In [3]:
# Usefull functions

def is_cube(a, p): 
    """Test if a is a cubic residue mod p, p prime"""
    return power_mod(a, (p-1)/gcd(3, p-1), p) == 1


def is_square(x, p): 
    """Test if x is a quadratic residue mod p, p prime"""
    return Mod(x, p).is_square()



def rand_prime_congru_1mod3_and_3mod4(size):
    """Randomly generate  a prime p, 2 <= p < 2^size with
       p = 1 mod 3 and p = 3 mod 4
    """
    p = random_prime(2 << size - 1)
    while (p%3 != 1) or (p%4 != 3):
        p = random_prime(2 << size - 1)
    return p


def rand_pq(size):
    """Randomly generate two distinct primes 
       p, q = 1 mod 3 and p, q = 3 mod 4
    """
    p = rand_prime_congru_1mod3_and_3mod4(size)
    q = p
    while q == p:
        q = rand_prime_congru_1mod3_and_3mod4(size)
    return (p, q)


# Solutions of quadratic equation ax^2+bx+c = 0 mod p^r using Hensel Lemma

def extend_solution_quadratic_equation_mod_pr(coeffs, sol, p, r):
    """Extend a solution *sol* from (mod p) to (mod p^r) of a quadratic equation ax^2+bx+c = 0 
       using Hensel Lemma
    """
    a, b, c = coeffs; x0 = int(sol)
    f = lambda x : a *x^2 + b*x + c
    fprime = lambda x : 2*a*x + b
    # We apply Hensel Lemma
    x0 = int(sol)
    for _ in range(2, r+1):
        x0 = (x0 - f(x0)*inverse_mod(fprime(x0), p^r)) % p^r
    return x0


def solver_quadratic_equation_mod_pr(coeffs, p, r):
    """Solutions of quadratic equation ax^2+bx+c = 0 (mod p^r) using Hensel Lemma"""
    a, b, c = coeffs; assert a%p != 0 and is_square(b^2-4*a*c, p)
    if  p%4 == 3:
        x1 = Mod((-b- power_mod(Mod(b^2-4*a*c, p), (p+1)//4, p)) * inverse_mod(2*a, p), p)
        x2 = Mod((-b + power_mod(Mod(b^2-4*a*c, p), (p+1)//4, p)) * inverse_mod(2*a, p), p)
    else:
        x1 = Mod((-b - Mod(b^2-4*a*c, p).sqrt()) * inverse_mod(2*a, p), p)
        x2 = Mod((-b + Mod(b^2-4*a*c, p).sqrt()) * inverse_mod(2*a, p), p)
        
    x1r = extend_solution_quadratic_equation_mod_pr(coeffs, x1, p, r)
    x2r = extend_solution_quadratic_equation_mod_pr(coeffs, x2, p, r)
    return (x1r, x2r)


def get_sol_with_cubic_pell_point_mod_pr_qs(C, p, q, r, s):
    """Determine the solutions mod p^r and mod q^s given a point 
       on the cubic Pell equation
    """
    xc, yc, zc = C; (a, b, c) = zc^3, yc^3-3*xc*yc*zc, xc^3 - 1
    sol_mod_pr = solver_quadratic_equation_mod_pr((a, b, c), p, r)
    sol_mod_qs = solver_quadratic_equation_mod_pr((a, b, c), q, s)
    return (sol_mod_pr, sol_mod_qs)


def is_solution_mod_pr(C, sol, p, r):
    """Test if *sol* is a solution of xc^3 + x*yc^3 + x^2 * zc^3 - 3x*xc*yc*zc = 1 mod p^r"""
    xc, yc, zc = C; (a, b, c) = zc^3, yc^3-3*xc*yc*zc, xc^3 - 1
    f = lambda x : a *x^2 + b*x + c
    return (f(sol) % p^r) == 0


def get_sols_with_ctheo(xp, yq, p, q, r, s):
    """ Determine a solution Y mod N with N=p^rq^s where
        Y = xp mod p^r
        Y = yq mod q^s
    """
    N = p^r * q^s
    X = (xp*q^s*inverse_mod(q^s, p^r) + yq*p^r*inverse_mod(p^r, q^s))
    return X % N


def get_possible_a_values(C, p, q, r, s):
    """Determine all values of a such that 
       
       xc^3 + a*yc^3 + a^2 * zc^3 - 3a*xc*yc*zc = 1 mod N with N = p^rq^s
       
    """
    N = p^r * q^s; (xc, yc, zc) = C
    sol_mod_pr, sol_mod_qs = get_sol_with_cubic_pell_point_mod_pr_qs(C, p, q, r, s)
    a1 =  get_sols_with_ctheo(sol_mod_pr[0], sol_mod_qs[0], p, q, r, s)
    a2 =  get_sols_with_ctheo(sol_mod_pr[1], sol_mod_qs[1], p, q, r, s)
    a3 =  get_sols_with_ctheo(sol_mod_pr[0], sol_mod_qs[1], p, q, r, s)
    a4 =  get_sols_with_ctheo(sol_mod_pr[1], sol_mod_qs[0], p, q, r, s)
    possible_a = (a1, a2, a3, a4)
    
    return possible_a


# Arithmetic of cubic pell curve

def add(pt1, pt2, a, N):
    """Add two points pt1 and pt2 on the cubic Pell curve
       C_a(N) = x^3+ay^3+a^2z^3-3axyz =  1 in Z/NZ 
    """
    (x1, y1, z1) =  pt1; (x2, y2, z2) = pt2
    x3 = (x1*x2 + a*(y2*z1 + y1*z2)) % N
    y3 = (x2*y1 + x1*y2 + a*z1*z2) % N
    z3 = (y1*y2 + x2*z1 + x1*z2) % N
    return (x3, y3, z3)


def exponent(pt, e, a, N):
    """Lft to right scalar multiplication on cubic Pell curve
       C_a(N) = x^3+ay^3+a^2z^3-3axyz =  1 in Z/NZ 
    """
    (x1, y1, z1) = pt; (x2, y2, z2) = (1, 0, 0)
    for bit in bin(e)[2:]:
        (x2, y2, z2) = add((x2, y2, z2), (x2, y2, z2), a, N)
        if bit == '1': 
            (x2, y2, z2) = add((x2, y2, z2), (x1, y1, z1), a, N)
    
    return (x2, y2, z2)

            
# Our Scheme 

def keygen(size, r, s):
    """The key generation algorithm
    Parameters:
    - :size: is an integer such that 2 <= p, q < 2^size
    """
    p, q = rand_pq(size)
    N = p^r * q^s
    ordn = p*q*(p^2+p+1)*(q^2+q+1)*(p - 1)^2*(q - 1)^2
    e = randint(1, N)
    while gcd(e, ordn) != 1:
        e = randint(1, N)
        
    d1 = inverse_mod(e, p^(2*(r-1))*q^(2*(s-1))*(p^2+p+1)*(q^2+q+1))
    d2 = inverse_mod(e, p^(2*(r-1))*q^(2*(s-1))*(p - 1)^2*(q - 1)^2)
    d3 = inverse_mod(e, p^(2*(r-1))*q^(2*(s-1))*(p^2+p+1)*(q - 1)^2)
    d4 = inverse_mod(e, p^(2*(r-1))*q^(2*(s-1))*(p - 1)^2*(q^2+q+1))
    return (N, e), (p, q, (r, s), (d1, d2, d3, d4))

    
def encipher(m, pk):
    """Compute the ciphertext of m using the public key pk"""
    (N, e) = pk; (xm, ym) = m
    a = ((1 - xm^3) * inverse_mod(ym^3, N)) % N
    C = exponent((xm, ym, 0), e, a, N)
    return C
    
    
def decipher(ctx, sk):
    """Decipher the ciphertext ctx using the secret key sk"""
    
    (p, q, (r, s), (d1, d2, d3, d4)) = sk
    N =  p^r*q^s
    possible_a = get_possible_a_values(ctx, p, q, r, s)
    for a in possible_a:
        if (not is_cube(a, p)) and (not is_cube(a, q)): 
            d = d1
        elif is_cube(a, p) and is_cube(a, q): 
            d = d2
        elif (not is_cube(a, p)) and is_cube(a, q): 
            d = d3
        else:
            d = d4
        (xm, ym, zm) = exponent(ctx, d, a, N)
        if zm == 0:
            return (xm, ym)


In [4]:
# Some Tests

r, s = 3, 2
pk, sk = keygen(40, r, s)
N = pk[0]
print("pk = ", pk, "sk = ", sk)

pk =  (69183613403715314293158976471566234696647195621131427425471, 23106690300302391939509400707029121671970426793992711357887) sk =  (965770804519, 277134447787, (3, 2), (1863493589498199108124227352386624487164260810028504221257791573429698621296452596498441611197550586851833735258830077, 4265329590040783123210603376115497248495137676144551411059130572911775393707150455246372385039407051682023196161930223, 2211385035869840099867582309576417035667901622732772419941711953012969236546310414593112475541957690964599580731691875, 2947686423734632693439405339355471360037582565631997855056805585658453660829819000755760789974478388424450877818697811))


In [5]:
number_ciphertext = 10
i = 1
while i <= number_ciphertext:
    mi = (randint(0, N), randint(0, N)); print(f"\nmi{i}\t      = {mi}", end="\n")
    ctx = encipher(mi, pk); print(f"ctx{i} = E(mi{i}) = {ctx}", end="\n")
    ptx = decipher(ctx, sk=sk); print(f"ptx{i} = D(ctx{i})= {ptx}")
    i += 1


mi1	      = (42643191398808801143705176507784190158672339680513316598435, 59078768134943148026852301058903034935429907062821596667303)
ctx1 = E(mi1) = (16568036415660199479908538528704273586397806052166910395154, 13464238732374805915346687744060912658496465140619153441947, 41601232817169863178085592933282324205119657813461942716339)
ptx1 = D(ctx1)= (42643191398808801143705176507784190158672339680513316598435, 59078768134943148026852301058903034935429907062821596667303)

mi2	      = (43206035556370981687524394457421930905481396633591233457952, 36677227345271311060203400530360617726438942899442137475483)
ctx2 = E(mi2) = (28163966204012869407145891489806587787603022485790566862832, 55927728631945545417992185099779585097585663352576746798388, 40948898327873233816398165898002970547420587211081511225289)
ptx2 = D(ctx2)= (43206035556370981687524394457421930905481396633591233457952, 36677227345271311060203400530360617726438942899442137475483)

mi3	      = (116046578981759078505271444916322173